# AdaptiveRegimeStrategy V4

In [142]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from statsmodels.tsa.stattools import adfuller
from abc import ABC, abstractmethod
import warnings
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')

## Feature Lab

In [143]:
class FeatureLab:
    """Shared mathematical engine for technical and statistical features."""
    
    @staticmethod
    def get_weights_frac_diff(d, size, threshold=1e-5):
        w = [1.0]
        for k in range(1, size):
            w_k = -w[-1] / k * (d - k + 1)
            w.append(w_k)
        w = np.array(w[::-1])
        w = w[np.abs(w) > threshold]
        return w

    @staticmethod
    def frac_diff_fixed(series, d, window=50):
        # Solves Stationarity Dilemma [cite: 61]
        weights = FeatureLab.get_weights_frac_diff(d, window)
        res = series.rolling(window=len(weights)).apply(lambda x: np.dot(x, weights), raw=True)
        return res

    @staticmethod
    def yang_zhang_volatility(df, window=30):
        # Captures intraday energy/gaps [cite: 82]
        log_ho = (df['High'] / df['Open']).apply(np.log)
        log_lo = (df['Low'] / df['Open']).apply(np.log)
        log_co = (df['Close'] / df['Open']).apply(np.log)
        log_oc = (df['Open'] / df['Close'].shift(1)).apply(np.log)
        log_cc = (df['Close'] / df['Close'].shift(1)).apply(np.log)
        
        rs = log_ho * (log_ho - log_co) + log_lo * (log_lo - log_co)
        close_vol = log_cc.rolling(window=window).var()
        open_vol = log_oc.rolling(window=window).var()
        window_rs = rs.rolling(window=window).mean()

        k = 0.34 / (1.34 + (window + 1) / (window - 1))
        return np.sqrt(open_vol + k * window_rs)

    @staticmethod
    def compute_rsi(series, window=14):
        delta = series.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

    @staticmethod
    def triple_barrier_labels(prices, vol, pt=1.0, sl=1.0, barrier_window=10):
        """
        Implements the Triple Barrier Method.
        Labels: 1 (Profit Target Hit), -1 (Stop Loss Hit), 0 (Time Limit/Neutral)
        """
        labels = pd.Series(0, index=prices.index)
        # Shift prices to align future outcome with current row
        # However, to avoid look-ahead in features, we usually compute label for row t based on t+1...t+k
        # This function generates the TARGET variable (y) for training.
        
        limit = len(prices) - barrier_window
        p_values = prices.values
        v_values = vol.values
        
        for i in range(limit):
            current_p = p_values[i]
            current_vol = v_values[i]
            
            # Dynamic barriers based on volatility [cite: 215]
            target = current_p * (1 + pt * current_vol)
            stop = current_p * (1 - sl * current_vol)
            
            future_window = p_values[i+1 : i+1+barrier_window]
            
            hit_target = np.where(future_window >= target)[0]
            hit_stop = np.where(future_window <= stop)[0]
            
            first_target = hit_target[0] if len(hit_target) > 0 else barrier_window + 1
            first_stop = hit_stop[0] if len(hit_stop) > 0 else barrier_window + 1
            
            if first_target < first_stop and first_target <= barrier_window:
                labels.iloc[i] = 1
            elif first_stop < first_target and first_stop <= barrier_window:
                labels.iloc[i] = 0 # In Meta-Labeling, we often treat Stop (-1) as 0 (Do Not Trade)
            # Else 0 (Time limit reached or neutral)
            
        return labels

## Base Strategy

In [144]:
class BaseStrategy(ABC):
    def __init__(self, ticker, start_date, end_date):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        self.data = None
        self.results = None
        self.metrics = {}

    def fetch_data(self, warmup_years=2):
        start_dt = datetime.strptime(self.start_date, "%Y-%m-%d")
        warmup_start_dt = start_dt - timedelta(days=warmup_years*365)
        warmup_start_str = warmup_start_dt.strftime("%Y-%m-%d")
        
        try:
            self.data = yf.download(self.ticker, start=warmup_start_str, end=self.end_date, progress=False, auto_adjust=False)
            if isinstance(self.data.columns, pd.MultiIndex): 
                self.data.columns = self.data.columns.get_level_values(0)
            if 'Adj Close' not in self.data.columns: 
                self.data['Adj Close'] = self.data['Close']
            self.data['Returns'] = self.data['Adj Close'].pct_change()
            self.data.dropna(inplace=True)
        except Exception as e:
            print(f"Error fetching {self.ticker}: {e}")
            self.data = pd.DataFrame()

    @abstractmethod
    def generate_signals(self):
        pass

    def run_backtest(self, transaction_cost=0.0005, rebalance_threshold=0.1):
        if self.data is None or self.data.empty: return
        
        backtest_mask = self.data.index >= self.start_date
        df = self.data.loc[backtest_mask].copy()
        if df.empty: return

        # Position Smoothing
        clean_positions = []
        current_pos = 0.0
        raw_signals = df['Signal'].values
        
        for target in raw_signals:
            if abs(target - current_pos) > rebalance_threshold:
                current_pos = target
            clean_positions.append(current_pos)
            
        df['Position'] = clean_positions
        df['Prev_Position'] = df['Position'].shift(1).fillna(0)
        df['Turnover'] = (df['Prev_Position'] - df['Position'].shift(2).fillna(0)).abs()
        df['Gross_Returns'] = df['Prev_Position'] * df['Returns']
        df['Net_Returns'] = df['Gross_Returns'] - (df['Turnover'] * transaction_cost)
        df['Net_Returns'].fillna(0, inplace=True)
        
        df['Cumulative_Strategy'] = (1 + df['Net_Returns']).cumprod()
        df['Cumulative_Market'] = (1 + df['Returns']).cumprod()
        
        roll_max = df['Cumulative_Strategy'].cummax()
        df['Drawdown'] = (df['Cumulative_Strategy'] / roll_max) - 1.0
        
        self.results = df
        
        # Performance Calculation
        total_ret = df['Cumulative_Strategy'].iloc[-1] - 1
        vol = df['Net_Returns'].std() * np.sqrt(252)
        sharpe = (df['Net_Returns'].mean() / df['Net_Returns'].std()) * np.sqrt(252) if vol > 0 else 0
        max_dd = df['Drawdown'].min()
        
        self.metrics = {
            'Total Return': total_ret,
            'Sharpe Ratio': sharpe,
            'Max Drawdown': max_dd
        }
        return df

## Model From NB V1

### V1_Baseline

In [145]:
class StrategyV1_Baseline(BaseStrategy):
    """V1: Fixed FracDiff, Standard GMM."""
    def generate_signals(self):
        if self.data is None or self.data.empty: return
        df = self.data.copy()
        
        df['Volatility'] = FeatureLab.yang_zhang_volatility(df)
        df['FracDiff'] = FeatureLab.frac_diff_fixed(df['Adj Close'].apply(np.log), d=0.4, window=50)
        df['RSI'] = FeatureLab.compute_rsi(df['Adj Close'])
        df['Returns_Smoothed'] = df['Returns'].rolling(5).mean()
        df['Vol_Smoothed'] = df['Volatility'].rolling(5).mean()
        df.dropna(inplace=True)
        
        X = df[['Returns_Smoothed', 'Vol_Smoothed']].values
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        gmm = GaussianMixture(n_components=3, random_state=42)
        df['Cluster'] = gmm.fit_predict(X_scaled)
        
        stats = df.groupby('Cluster')['Returns_Smoothed'].mean().sort_values().index
        mapping = {stats[0]: -1, stats[1]: 0, stats[2]: 1}
        df['Regime'] = df['Cluster'].map(mapping)
        
        df['Signal'] = 0
        df.loc[(df['Regime'] == 1) & (df['FracDiff'] > 0), 'Signal'] = 1
        df.loc[(df['Regime'] == 0) & (df['RSI'] < 40), 'Signal'] = 1
        
        target_vol = 0.15 / np.sqrt(252)
        df['Vol_Scaler'] = (target_vol / df['Volatility']).clip(upper=1.5)
        df['Signal'] = df['Signal'] * df['Vol_Scaler']
        self.data = df

### V3_Macro

In [146]:
class StrategyV3_Macro(BaseStrategy):
    """V3: Macro (SPY) Filter."""
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        self.spy_data = None

    def fetch_data(self, warmup_years=2):
        super().fetch_data(warmup_years)
        start_dt = datetime.strptime(self.start_date, "%Y-%m-%d") - timedelta(days=warmup_years*365)
        try:
            spy = yf.download("SPY", start=start_dt.strftime("%Y-%m-%d"), end=self.end_date, progress=False, auto_adjust=False)
            if isinstance(spy.columns, pd.MultiIndex): spy.columns = spy.columns.get_level_values(0)
            self.spy_data = spy[['Adj Close']].rename(columns={'Adj Close': 'SPY_Price'})
        except: pass

    def generate_signals(self):
        if self.data is None or self.data.empty: return
        df = self.data.copy()
        
        if self.spy_data is not None:
            df = df.join(self.spy_data, how='left')
            df['SPY_MA200'] = df['SPY_Price'].rolling(window=200).mean()
            df['Macro_Bull'] = df['SPY_Price'] > df['SPY_MA200']
        else:
            df['Macro_Bull'] = True
            
        df['Volatility'] = FeatureLab.yang_zhang_volatility(df)
        df['FracDiff'] = FeatureLab.frac_diff_fixed(df['Adj Close'].apply(np.log), d=0.4, window=50)
        df['RSI'] = FeatureLab.compute_rsi(df['Adj Close'])
        df.dropna(inplace=True)
        
        df['Signal'] = 0
        df.loc[(df['FracDiff'] > 0), 'Signal'] = 1
        df.loc[df['Macro_Bull'] == False, 'Signal'] = 0
        
        target_vol = 0.15 / np.sqrt(252)
        df['Signal'] = df['Signal'] * (target_vol / df['Volatility']).clip(upper=1.5)
        self.data = df

### V9_Unshackled

In [147]:
class StrategyV9_RegimeUnshackled(BaseStrategy):
    """
    V9 (Robust): The 'Unshackled' Regime Model with Walk-Forward Learning.
    
    CRITICAL FIX:
    - Replaces static gmm.fit_predict() with a Rolling Walk-Forward loop.
    - Eliminates Look-Ahead Bias by retraining the regime model every month
      using only the trailing 1-year window.
    - Solves the 'Label Switching' problem dynamically at each step.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        self.spy_data = None

    def fetch_data(self, warmup_years=2):
        super().fetch_data(warmup_years)
        start_dt = datetime.strptime(self.start_date, "%Y-%m-%d") - timedelta(days=warmup_years*365)
        try:
            # Fetch Macro Context (SPY) for the 'Alpha Clause'
            spy = yf.download("SPY", start=start_dt.strftime("%Y-%m-%d"), end=self.end_date, progress=False, auto_adjust=False)
            if isinstance(spy.columns, pd.MultiIndex): spy.columns = spy.columns.get_level_values(0)
            
            # Simple Macro Trend (200 SMA)
            spy['Macro_Trend'] = (spy['Adj Close'] > spy['Adj Close'].rolling(200).mean()).astype(int)
            self.spy_data = spy[['Macro_Trend']]
        except: 
            pass

    def _apply_kalman_filter(self, prices):
        # [cite: 151] Kalman Filter for recursive state estimation
        x = prices.values
        n = len(x)
        state = np.zeros(n)
        slope = np.zeros(n)
        state[0] = x[0]
        P, Q, R = 1.0, 0.001, 0.1 # Static params for robustness
        
        for t in range(1, n):
            pred_state = state[t-1] + slope[t-1]
            pred_P = P + Q
            measurement = x[t]
            residual = measurement - pred_state
            
            K = pred_P / (pred_P + R)
            state[t] = pred_state + K * residual
            slope[t] = 0.9 * slope[t-1] + 0.1 * (state[t] - state[t-1])
            P = (1 - K) * pred_P
            
        return pd.Series(slope, index=prices.index)

    def generate_signals(self):
        if self.data is None or self.data.empty: return
        df = self.data.copy()
        
        # --- 1. Feature Engineering ---
        if self.spy_data is not None:
            df = df.join(self.spy_data, how='left').fillna(method='ffill')
        else: 
            df['Macro_Trend'] = 1
            
        # Volatility & Kalman Slope [cite: 82, 151]
        df['Volatility'] = FeatureLab.yang_zhang_volatility(df)
        df['Kalman_Slope'] = self._apply_kalman_filter(np.log(df['Adj Close']))
        df['RSI'] = FeatureLab.compute_rsi(df['Adj Close'])
        
        # Features for GMM (Smoothed to reduce noise)
        df['Returns_Smoothed'] = df['Returns'].rolling(5).mean()
        df['Vol_Smoothed'] = df['Volatility'].rolling(5).mean()
        
        # Drop NaN from warmup
        df.dropna(inplace=True)
        
        # --- 2. Rolling Walk-Forward GMM ---
        # We need to predict 'Regime_Type' for every row without looking ahead.
        
        train_window = 252  # Lookback: 1 Year
        refit_step = 21     # Re-train every Month
        
        # Initialize default regime (CHOP)
        df['Regime_Type'] = 'CHOP' 
        
        # Prepare Feature Matrix
        X = df[['Returns_Smoothed', 'Vol_Smoothed']].values
        scaler = StandardScaler()
        
        indices = np.arange(len(df))
        
        # Start loop after the first training window
        for t in range(train_window, len(df), refit_step):
            start_idx = t - train_window
            end_idx = t
            predict_end_idx = min(t + refit_step, len(df))
            
            # A. Train on PAST window
            X_train = X[start_idx:end_idx]
            
            # Scale locally (Standardization must also be walk-forward)
            #  "Standard algorithms assume training data... same probability distribution"
            scaler_local = StandardScaler()
            X_train_scaled = scaler_local.fit_transform(X_train)
            
            try:
                # Fit GMM
                gmm = GaussianMixture(n_components=3, random_state=42, n_init=5)
                gmm.fit(X_train_scaled)
                
                # B. Predict on NEXT window (Future)
                X_future = X[end_idx:predict_end_idx]
                X_future_scaled = scaler_local.transform(X_future)
                clusters_future = gmm.predict(X_future_scaled)
                
                # C. Dynamic Label Mapping (Solve Label Switching)
                # We map clusters to Bull/Bear based on the *Training* means
                # Calculate mean return for each cluster center in the training set
                # We can approximate this by predicting the training set again
                train_clusters = gmm.predict(X_train_scaled)
                
                # Create a temp dataframe to sort clusters
                temp_df = pd.DataFrame({
                    'Ret': X_train[:, 0], # Returns is column 0
                    'Cluster': train_clusters
                })
                
                stats = temp_df.groupby('Cluster')['Ret'].mean().sort_values()
                
                # The cluster with lowest mean return = BEAR
                # The cluster with highest mean return = BULL
                # Middle = CHOP
                if len(stats) == 3:
                    bear_c = stats.index[0]
                    chop_c = stats.index[1]
                    bull_c = stats.index[2]
                    
                    mapping = {bear_c: 'BEAR', chop_c: 'CHOP', bull_c: 'BULL'}
                else:
                    # Fallback if GMM collapses to fewer clusters
                    mapping = {c: 'CHOP' for c in stats.index}

                # Apply mapping to the prediction window
                regimes_mapped = [mapping.get(c, 'CHOP') for c in clusters_future]
                
                # Store results in the main DataFrame
                # Use iloc for integer-based indexing on the slice
                df.iloc[end_idx:predict_end_idx, df.columns.get_loc('Regime_Type')] = regimes_mapped
                
            except Exception as e:
                # Keep default 'CHOP' on failure
                pass

        # --- 3. Unshackled Signal Logic (unchanged from V9) ---
        df['Signal'] = 0.0
        
        # A. BULL REGIME: "Trust The Trend"
        bull_signal = (df['Regime_Type'] == 'BULL')
        df.loc[bull_signal, 'Signal'] = 1.0
        
        # Boost: If Kalman agrees (Strong Trend), increase leverage
        strong_trend = bull_signal & (df['Kalman_Slope'] > 0)
        df.loc[strong_trend, 'Signal'] = 1.3
        
        # B. CHOP REGIME: "Buy Dips"
        # chop_buy = (df['Regime_Type'] == 'CHOP') & (df['RSI'] < 45)
        # df.loc[chop_buy, 'Signal'] = 1.0

        # 1. Calculate a "Floor" (e.g., Lower Bollinger Band)
        df['SMA_20'] = df['Adj Close'].rolling(20).mean()
        df['BB_Lower'] = df['SMA_20'] - 2 * df['SMA_20'].rolling(20).std()

        # 2. Strict Chop Entry
        # OLD: RSI < 45
        # NEW: RSI < 45 AND Price > BB_Lower (Don't buy if it's crashing through the floor)
        #      AND FracDiff > -0.1 (Don't buy if trend is catastrophically negative)

        chop_buy = (
            (df['Regime_Type'] == 'CHOP') & 
            (df['RSI'] < 45) & 
            (df['Adj Close'] > df['BB_Lower']) # The Falling Knife Filter
        )
        df.loc[chop_buy, 'Signal'] = 1.0
        
        # C. BEAR REGIME: "Survival" (Cash unless deep panic)
        panic_buy = (df['Regime_Type'] == 'BEAR') & (df['RSI'] < 25)
        df.loc[panic_buy, 'Signal'] = 1.0
        
        # --- 4. The Alpha Clause (Macro Handling) ---
        target_vol = 0.15 / np.sqrt(252)
        
        # Volatility Sizing (Risk Parity) [cite: 254]
        vol_scaler = (target_vol / df['Volatility']).clip(upper=1.5)
        
        # Macro Filter: Halve size if SPY is bearish, but don't exit fully (Alpha Clause)
        macro_scaler = df['Macro_Trend'].map({1: 1.0, 0: 0.5})
        
        df['Signal'] = df['Signal'] * vol_scaler * macro_scaler
        
        self.data = df

### Ensemble

In [148]:
class Strategy_Ensemble(BaseStrategy):
    """
    The 'All-Weather' Ensemble.
    
    Combines V3 (Macro Trend) and V9 (Regime Unshackled) into a single
    portfolio-level signal.
    
    Logic:
    1. Runs V3 to capture high-beta trends (NVDA, Bitcoin).
    2. Runs V9 to capture regime-based alpha and protect downside (JPM, BABA).
    3. Blends signals using a 'Correlation-Adjusted' weighting or fixed 50/50.
    4. Applies a final Volatility Target to the combined equity curve to ensure
       the two strategies don't stack up to dangerous leverage.
    """
    def __init__(self, ticker, start_date, end_date, w_v3=0.5, w_v9=0.5):
        super().__init__(ticker, start_date, end_date)
        self.w_v3 = w_v3
        self.w_v9 = w_v9
        # Instantiate sub-strategies
        self.strat_v3 = StrategyV3_Macro(ticker, start_date, end_date)
        self.strat_v9 = StrategyV9_RegimeUnshackled(ticker, start_date, end_date)

    def fetch_data(self, warmup_years=2):
        # Fetch once for efficiency (logic could be optimized to share DF, 
        # but separate fetch ensures cleaner encapsulation)
        self.strat_v3.fetch_data(warmup_years)
        self.strat_v9.fetch_data(warmup_years)
        
        # We share the index/data from one of them for the main wrapper
        if self.strat_v3.data is not None and not self.strat_v3.data.empty:
            self.data = self.strat_v3.data.copy()
        elif self.strat_v9.data is not None:
            self.data = self.strat_v9.data.copy()

    def generate_signals(self):
        if self.strat_v3.data is None or self.strat_v9.data is None: return
        
        # 1. Generate Sub-Signals
        self.strat_v3.generate_signals()
        self.strat_v9.generate_signals()
        
        # Align Indices (Inner Join to be safe)
        df = self.data.copy()
        s3 = self.strat_v3.data['Signal']
        s9 = self.strat_v9.data['Signal']
        
        # Merge signals into main DF
        df['Sig_V3'] = s3
        df['Sig_V9'] = s9
        df.dropna(inplace=True)
        
        # 2. The Allocation Logic
        # Default: Fixed Weight (Core-Satellite Approach)
        # V3 (Beta) + V9 (Alpha)
        
        # We blend the RAW signals.
        # Note: Signals are already Vol-Targeted to ~15% inside sub-classes.
        # Simple addition would double vol if correlation=1.
        raw_blend = (df['Sig_V3'] * self.w_v3) + (df['Sig_V9'] * self.w_v9)
        
        # 3. Ensemble Volatility Control
        # If V3 and V9 agree (both Long), we get high exposure.
        # If they disagree (V3 Long, V9 Cash), we get half exposure.
        # This naturally deleverages during uncertainty.
        
        df['Signal'] = raw_blend
        
        # Optional: Re-Target Volatility of the *Ensemble*
        # (Prevents leverage creep if strategies are highly correlated)
        # For now, we trust the weighted sum to act as a diversification benefit.
        
        self.data = df

### Adaptive Ensemble

In [149]:
class Strategy_Ensemble_Adaptive(BaseStrategy):
    """
    V10: The Adaptive Ensemble (Dynamic Weighting).
    
    Instead of fixed weights, this strategy re-allocates capital quarterly 
    based on the recent Risk-Adjusted Performance (Sharpe) of the sub-strategies.
    
    Logic:
    1. Lookback: 126 Days (6 Months).
    2. Rebalance: Every 63 Days (Quarterly).
    3. Weighting:
       - Calculate Sharpe Ratio for V3 and V9 in the lookback window.
       - If Sharpe > 0: Weight is proportional to Sharpe.
       - If Sharpe < 0: Weight is set to 0.
       - Normalize weights to sum to 1.0.
       
    This allows the portfolio to automatically 'Risk On' into V3 during strong bulls
    and 'Risk Off' into V9 during bears/chop.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        # Instantiate sub-strategies
        self.strat_v3 = StrategyV3_Macro(ticker, start_date, end_date)
        self.strat_v9 = StrategyV9_RegimeUnshackled(ticker, start_date, end_date)

    def fetch_data(self, warmup_years=2):
        self.strat_v3.fetch_data(warmup_years)
        self.strat_v9.fetch_data(warmup_years)
        
        # Use one of the dataframes as the base
        if self.strat_v3.data is not None and not self.strat_v3.data.empty:
            self.data = self.strat_v3.data.copy()
        elif self.strat_v9.data is not None:
            self.data = self.strat_v9.data.copy()

    def generate_signals(self):
        if self.strat_v3.data is None or self.strat_v9.data is None: return
        
        # 1. Generate Sub-Signals
        self.strat_v3.generate_signals()
        self.strat_v9.generate_signals()
        
        # Merge Data
        df = self.data.copy()
        df['Sig_V3'] = self.strat_v3.data['Signal']
        df['Sig_V9'] = self.strat_v9.data['Signal']
        df.dropna(inplace=True)
        
        # 2. Simulate Sub-Strategy Returns (for metric calculation)
        # We need to know how they *would* have performed to weight them.
        # Lag signals by 1 to avoid lookahead bias when calculating returns.
        df['Ret_V3'] = df['Sig_V3'].shift(1) * df['Returns']
        df['Ret_V9'] = df['Sig_V9'].shift(1) * df['Returns']
        
        # 3. Walk-Forward Weight Optimization
        df['W_V3'] = 0.5 # Default start
        df['W_V9'] = 0.5
        
        lookback = 126      # 6 Months Lookback
        rebalance_freq = 21 # Monthly Rebalance (Faster adaptation)
        
        indices = df.index
        
        if len(df) > lookback:
            for t in range(lookback, len(df), rebalance_freq):
                train_start = indices[t - lookback]
                train_end = indices[t]
                test_end_idx = min(t + rebalance_freq, len(df))
                test_end = indices[test_end_idx - 1]
                
                # Calculate Sharpe in Lookback Window
                # Add small epsilon to std to avoid division by zero
                v3_mean = df.loc[train_start:train_end, 'Ret_V3'].mean()
                v3_std = df.loc[train_start:train_end, 'Ret_V3'].std() + 1e-9
                sharpe_v3 = (v3_mean / v3_std) * np.sqrt(252)
                
                v9_mean = df.loc[train_start:train_end, 'Ret_V9'].mean()
                v9_std = df.loc[train_start:train_end, 'Ret_V9'].std() + 1e-9
                sharpe_v9 = (v9_mean / v9_std) * np.sqrt(252)
                
                # Weighting Logic
                # 1. Filter: If Sharpe is negative, set score to 0
                score_v3 = max(0, sharpe_v3)
                score_v9 = max(0, sharpe_v9)
                
                # 2. Normalize
                total_score = score_v3 + score_v9
                
                if total_score > 0:
                    w_v3 = score_v3 / total_score
                    w_v9 = score_v9 / total_score
                else:
                    # Both are failing? Default to Defensive (V9) or Cash (0)
                    # Let's default to V9 (Safety) as the 'bunker'
                    w_v3 = 0.0
                    w_v9 = 1.0
                
                # Apply weights to NEXT window
                df.loc[train_end:test_end, 'W_V3'] = w_v3
                df.loc[train_end:test_end, 'W_V9'] = w_v9
                
        # 4. Final Signal Generation
        # Blend the signals using the dynamic weights
        df['Signal'] = (df['Sig_V3'] * df['W_V3']) + (df['Sig_V9'] * df['W_V9'])
        
        self.data = df

### V12_Macro

In [150]:
class StrategyV12_Macro_Switch(BaseStrategy):
    """
    V12: The Macro-Guided Ensemble.
    
    Replaces lookback windows with Real-Time Economic Data.
    
    DATA SOURCES (Yahoo Finance):
    1. ^VIX: CBOE Volatility Index.
    2. ^TNX: 10-Year Treasury Yield.
    
    LOGIC:
    1. Calculate 'Macro Stress Score' (0.0 to 1.0).
       - VIX Component: Normalized against recent history. High VIX = High Stress.
       - Yield Component: Rate of Change (ROC) of TNX. Spiking rates = High Stress.
    
    2. Dynamic Weighting:
       - Weight_V3 (Trend) = 1.0 - Stress_Score
       - Weight_V9 (Safety) = Stress_Score
       
    HYPOTHESIS:
    VIX and Rates often spike BEFORE the price crash is fully realized. 
    This allows the model to switch to safety faster than a Moving Average.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        self.v3 = StrategyV3_Macro(ticker, start_date, end_date)
        self.v9 = StrategyV9_RegimeUnshackled(ticker, start_date, end_date)
        # We store macro data separately
        self.macro_data = None

    def fetch_data(self, warmup_years=2):
        # 1. Fetch Ticker Data
        self.v3.fetch_data(warmup_years)
        self.v9.fetch_data(warmup_years)
        
        if self.v3.data is None or self.v3.data.empty: return
        self.data = self.v3.data.copy()
        
        # 2. Fetch Macro Data (VIX and TNX)
        start_dt = (self.data.index[0] - timedelta(days=365)).strftime("%Y-%m-%d")
        end_dt = self.end_date
        
        try:
            vix = yf.download("^VIX", start=start_dt, end=end_dt, progress=False, auto_adjust=True)
            tnx = yf.download("^TNX", start=start_dt, end=end_dt, progress=False, auto_adjust=True)
            
            # Cleaning
            if isinstance(vix.columns, pd.MultiIndex): vix.columns = vix.columns.get_level_values(0)
            if isinstance(tnx.columns, pd.MultiIndex): tnx.columns = tnx.columns.get_level_values(0)
            
            macro_df = pd.DataFrame(index=self.data.index)
            # Align macro data to the ticker's trading days (ffill for holidays)
            macro_df['VIX'] = vix['Close'].reindex(self.data.index, method='ffill')
            macro_df['TNX'] = tnx['Close'].reindex(self.data.index, method='ffill')
            
            self.macro_data = macro_df
            
        except Exception as e:
            print(f"Macro Data Fetch Error: {e}")
            # Fallback: Zero stress
            self.macro_data = pd.DataFrame({'VIX': 20, 'TNX': 4}, index=self.data.index)

    def generate_signals(self):
        if self.data is None or self.macro_data is None: return
        
        # 1. Run Sub-Strategies
        self.v3.generate_signals()
        self.v9.generate_signals()
        
        # 2. Sync Data
        df = self.data.copy()
        df = df.join(self.v3.data[['Signal']].rename(columns={'Signal':'S_V3'}), how='left')
        df = df.join(self.v9.data[['Signal']].rename(columns={'Signal':'S_V9'}), how='left')
        
        # 3. Calculate Macro Stress Score
        macro = self.macro_data.copy()
        
        # A. VIX Stress (Fear)
        # Normalize VIX: If VIX > 30, Stress = 1.0. If VIX < 15, Stress = 0.0.
        # Uses a rolling Z-score or simple clamp? Simple clamp is more robust to regime shifts.
        macro['VIX_Stress'] = ((macro['VIX'] - 15) / (30 - 15)).clip(0, 1)
        
        # B. Yield Stress (Rate Shock)
        # We care about SPEED of rate rise, not just level.
        # Calculate 20-day Rate of Change of TNX
        macro['TNX_ROC'] = macro['TNX'].pct_change(20)
        # If Yields rise > 10% in a month, that's a shock.
        macro['TNX_Stress'] = (macro['TNX_ROC'] / 0.10).clip(0, 1)
        
        # Combined Stress (Max of either Fear or Rate Shock)
        # We use Max because either one can crash the market independently.
        macro['Total_Stress'] = macro[['VIX_Stress', 'TNX_Stress']].max(axis=1)
        
        # 4. Allocate Weights
        # Smooth the stress signal to avoid daily jitter (3-day avg)
        stress_signal = macro['Total_Stress'].rolling(3).mean().fillna(0)
        
        df['W_V9'] = stress_signal        # High Stress -> More Safety
        df['W_V3'] = 1.0 - stress_signal  # Low Stress -> More Trend
        
        # 5. Final Signal
        df['Signal'] = (df['S_V3'] * df['W_V3']) + (df['S_V9'] * df['W_V9'])
        
        self.data = df

### V21_MacroMom

In [151]:
class StrategyV21_MacroMomentum(BaseStrategy):
    """
    V21: The Production Candidate.
    
    Architecture:
    1. BASE: V20 Macro Stress Logic (VIX + TNX).
    2. OVERRIDE: 'Trend Floor'. If the asset is structurally bullish 
       (Price > SMA200), we refuse to go to zero exposure, even if Macro is stressed.
       
    This solves the 'TSLA Problem' (missing rallies) without relying on 
    unstable Neural Networks.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        self.macro_data = None

    def fetch_data(self, warmup_years=2):
        super().fetch_data(warmup_years)
        
        # 1. Fetch Macro Data
        try:
            start_dt = self.data.index[0]
            vix = yf.download("^VIX", start=start_dt, end=self.end_date, progress=False, auto_adjust=True)
            tnx = yf.download("^TNX", start=start_dt, end=self.end_date, progress=False, auto_adjust=True)
            
            if isinstance(vix.columns, pd.MultiIndex): vix.columns = vix.columns.get_level_values(0)
            if isinstance(tnx.columns, pd.MultiIndex): tnx.columns = tnx.columns.get_level_values(0)
            
            self.macro_data = pd.DataFrame(index=self.data.index)
            self.macro_data['VIX'] = vix['Close'].reindex(self.data.index).fillna(method='ffill')
            self.macro_data['TNX'] = tnx['Close'].reindex(self.data.index).fillna(method='ffill')
        except:
            self.macro_data = pd.DataFrame({'VIX': 20, 'TNX': 4.0}, index=self.data.index)

    def generate_signals(self):
        if self.data is None or self.macro_data is None: return
        df = self.data.copy()
        macro = self.macro_data.copy()
        
        # --- 1. Macro Stress Engine (V20 Logic) ---
        # VIX Stress: Normalizing 12-35 range
        macro['VIX_Stress'] = ((macro['VIX'] - 12) / 23).clip(0, 1)
        
        # Rates Stress: 1-Month Rate of Change > 10% is Panic
        macro['TNX_ROC'] = macro['TNX'].pct_change(20)
        macro['TNX_Stress'] = (macro['TNX_ROC'] / 0.10).clip(0, 1)
        
        # Combined Stress (Smoothed)
        macro['Total_Stress'] = macro[['VIX_Stress', 'TNX_Stress']].max(axis=1).rolling(5).mean().fillna(0)
        
        # Base Signal: Inverse of Stress
        df['Base_Signal'] = 1.0 - macro['Total_Stress']
        
        # --- 2. The Momentum Override (The Fix) ---
        # "Don't fight the tape."
        
        # Long Term Trend (SMA 200)
        df['SMA_200'] = df['Adj Close'].rolling(200).mean()
        df['Trend_Bull'] = (df['Adj Close'] > df['SMA_200'])
        
        # Momentum (RSI)
        df['RSI'] = FeatureLab.compute_rsi(df['Adj Close'])
        df['Mom_Bull'] = (df['RSI'] > 50)
        
        # LOGIC: 
        # If Trend is BULL and Momentum is BULL, Minimum Signal = 0.5.
        # This ensures we participate in "Hated Rallies" (like TSLA 2023)
        # even if VIX is elevated.
        
        df['Signal'] = df['Base_Signal']
        
        # Apply Override
        strong_uptrend = df['Trend_Bull'] & df['Mom_Bull']
        df.loc[strong_uptrend, 'Signal'] = np.maximum(df.loc[strong_uptrend, 'Signal'], 0.5)
        
        # --- 3. Safety Veto (The "Falling Knife" Guard) ---
        # If Price is crashing < Lower Bollinger, KILL the trade.
        df['SMA_20'] = df['Adj Close'].rolling(20).mean()
        df['BB_Lower'] = df['SMA_20'] - 2 * df['SMA_20'].rolling(20).std()
        
        # Allow tiny buffer (0.98)
        is_crashing = df['Adj Close'] < (df['BB_Lower'] * 0.98)
        df.loc[is_crashing, 'Signal'] = 0.0

        # --- 4. Volatility Sizing ---
        target_vol = 0.15 / np.sqrt(252)
        vol_scaler = (target_vol / FeatureLab.yang_zhang_volatility(df)).clip(upper=1.5)
        
        df['Signal'] = df['Signal'] * vol_scaler
        
        self.data = df

### V21_MacroT_Portfolio

In [152]:
class Strategy_V21_Sector_Portfolio(BaseStrategy):
    """
    V21_Sector: Applies V21 Logic to the entire Sector Universe.
    
    This is the CONTROL strategy to fairly benchmark against TSR_V2.
    It allows V21 to rotate into Energy/Defensives naturally if the 
    Macro/Trend logic dictates it.
    """
    def __init__(self, ticker, start_date, end_date):
        # Ticker is ignored, we load the Sector Basket
        super().__init__(ticker, start_date, end_date)
        self.sectors = ['XLE', 'XLF', 'XLK', 'XLY', 'XLI', 'XLV', 'XLP', 'XLU', 'XLB']
        self.sector_data = None
        self.macro_data = None

    def fetch_data(self, warmup_years=2):
        start_dt = datetime.strptime(self.start_date, "%Y-%m-%d") - timedelta(days=warmup_years*365)
        start_str = start_dt.strftime("%Y-%m-%d")
        
        try:
            # 1. Fetch Sector Data
            sec_df = yf.download(self.sectors, start=start_str, end=self.end_date, progress=False, auto_adjust=True)
            if isinstance(sec_df.columns, pd.MultiIndex):
                self.sector_data = sec_df['Close'].fillna(method='ffill')
            else:
                self.sector_data = sec_df['Close'].fillna(method='ffill')

            # 2. Fetch Macro Data (For V21 Logic)
            macro_tickers = ['^VIX', '^TNX']
            macro_df = yf.download(macro_tickers, start=start_str, end=self.end_date, progress=False, auto_adjust=True)
            if isinstance(macro_df.columns, pd.MultiIndex):
                self.macro_data = macro_df['Close'].fillna(method='ffill')
            else:
                self.macro_data = macro_df['Close'].fillna(method='ffill')
                
            # Base Container
            self.data = pd.DataFrame(index=self.sector_data.index)
            self.data['Adj Close'] = 100.0
            
        except Exception as e:
            print(f"V21 Sector Fetch Error: {e}")

    def generate_signals(self):
        if self.sector_data is None or self.macro_data is None: return
        
        # --- 1. Macro Stress Engine (V21 Global Logic) ---
        macro = self.macro_data.copy()
        macro['VIX_Stress'] = ((macro['^VIX'] - 12) / 23).clip(0, 1)
        macro['TNX_ROC'] = macro['^TNX'].pct_change(20)
        macro['TNX_Stress'] = (macro['TNX_ROC'] / 0.10).clip(0, 1)
        macro['Total_Stress'] = macro[['VIX_Stress', 'TNX_Stress']].max(axis=1).rolling(5).mean().fillna(0)
        
        base_signal_strength = 1.0 - macro['Total_Stress']
        
        # --- 2. Sector-Level Logic Loop ---
        # We calculate the V21 signal for EACH sector individually
        active_weights = pd.DataFrame(0.0, index=self.sector_data.index, columns=self.sectors)
        
        for sector in self.sectors:
            prices = self.sector_data[sector]
            
            # A. Trend Logic (SMA 200)
            sma200 = prices.rolling(200).mean()
            trend_bull = (prices > sma200)
            
            # B. Momentum Logic (RSI)
            delta = prices.diff()
            gain = (delta.where(delta > 0, 0)).rolling(14).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
            rs = gain / loss
            rsi = 100 - (100 / (1 + rs))
            mom_bull = (rsi > 50)
            
            # C. Falling Knife Logic (Bollinger Lower)
            sma20 = prices.rolling(20).mean()
            std20 = prices.rolling(20).std()
            bb_lower = sma20 - 2 * std20
            is_crashing = prices < (bb_lower * 0.98)
            
            # D. Combine
            # Default = Base Signal (Macro Driven)
            sig = base_signal_strength.copy()
            
            # Override: If Trend+Mom are Bullish, minimum exposure 0.5 (Ignore Macro Fear)
            strong_uptrend = trend_bull & mom_bull
            sig[strong_uptrend] = np.maximum(sig[strong_uptrend], 0.5)
            
            # Veto: If Crashing, Zero exposure
            sig[is_crashing] = 0.0
            
            active_weights[sector] = sig

        # --- 3. Portfolio Construction ---
        # Equal Weight across all active signals
        # If 5 sectors have signal 1.0, we hold 20% of each.
        # If all 9 have signal 1.0, we hold 11% of each.
        
        # We normalize row-wise so leverage doesn't exceed 1.0? 
        # Or do we treat each as an independent bet?
        # Let's Normalize to 1.0 leverage to be fair to TSR (which is usually L/S or 100% Long).
        
        total_signal = active_weights.sum(axis=1)
        # Avoid division by zero
        scale_factor = (1.0 / total_signal).replace([np.inf, -np.inf], 0.0).clip(0, 1)
        
        # Final Portfolio Weights
        final_weights = active_weights.multiply(scale_factor, axis=0)
        
        # Calculate Returns
        # Portfolio Return = Sum(Weight_i * Return_i)
        sec_rets = self.sector_data.pct_change().fillna(0)
        port_ret = (final_weights * sec_rets).sum(axis=1)
        
        self.data['Returns'] = port_ret
        self.data['Net_Returns'] = port_ret
        self.data['Signal'] = 1 # Dummy

### Max Potential

In [153]:
class Strategy_CrystalBall(BaseStrategy):
    """
    Theoretical Maximum: Perfect Foresight.
    
    Logic:
    At the close of Day T, look at Day T+1 returns.
    - If the best performing sector is > 0%: Buy 100% of that sector.
    - If all sectors are negative: Hold Cash (0% return).
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        self.sectors = ['XLE', 'XLF', 'XLK', 'XLY', 'XLI', 'XLV', 'XLP', 'XLU', 'XLB']
        self.sector_data = None

    def fetch_data(self, warmup_years=0):
        # Fetch Sectors
        try:
            start_dt = datetime.strptime(self.start_date, "%Y-%m-%d") - timedelta(days=30) # Minimal warmup
            sec_df = yf.download(self.sectors, start=start_dt.strftime("%Y-%m-%d"), end=self.end_date, progress=False, auto_adjust=True)
            
            if isinstance(sec_df.columns, pd.MultiIndex):
                self.sector_data = sec_df['Close'].fillna(method='ffill')
            else:
                self.sector_data = sec_df['Close'].fillna(method='ffill')
                
            self.data = pd.DataFrame(index=self.sector_data.index)
            self.data['Adj Close'] = 100.0
            
        except Exception as e:
            print(f"Crystal Ball Error: {e}")

    def generate_signals(self):
        # Calculate Forward Returns (Lookahead Bias Intentional)
        # Shift(-1) brings tomorrow's return to today.
        rets = self.sector_data.pct_change().shift(-1).fillna(0)
        
        # Identify the Max Return for tomorrow
        max_ret = rets.max(axis=1)
        
        # Strategy Return:
        # If Max Ret > 0, we capture it.
        # If Max Ret < 0, we stay in cash (0.0).
        self.data['Net_Returns'] = np.where(max_ret > 0, max_ret, 0.0)

    def run_backtest(self, transaction_cost=0.0, rebalance_threshold=0.0):
        # Custom backtest for metrics
        if self.data is None or self.data.empty: return
        backtest_mask = self.data.index >= self.start_date
        df = self.data.loc[backtest_mask].copy()
        
        df['Cumulative_Strategy'] = (1 + df['Net_Returns']).cumprod()
        roll_max = df['Cumulative_Strategy'].cummax()
        df['Drawdown'] = (df['Cumulative_Strategy'] / roll_max) - 1.0
        self.results = df
        
        total_ret = df['Cumulative_Strategy'].iloc[-1] - 1
        vol = df['Net_Returns'].std() * np.sqrt(252)
        sharpe = (df['Net_Returns'].mean() / df['Net_Returns'].std()) * np.sqrt(252) if vol > 0 else 0
        max_dd = df['Drawdown'].min()
        
        self.metrics = {'Total Return': total_ret, 'Sharpe Ratio': sharpe, 'Max Drawdown': max_dd}
        return df

## New V4 Models

### V22_SortioUnbound

In [154]:
class StrategyV22_Sortino_Unbound(BaseStrategy):
    """
    V22: The 'Sortino' Strategy (Fixes the NVDA/XLE Underperformance).
    
    CRITICAL IMPROVEMENT:
    Standard volatility targeting (VolTarget / TotalVol) penalizes "Good Volatility" 
    (Upside breakouts). When NVDA goes parabolic, standard deviation explodes, 
    causing V3/V9 to cut position size.
    
    V22 replaces Total Volatility with 'Downside Deviation' for position sizing.
    Logic:
    1. Calculate Downside Deviation (volatility of negative returns only).
    2. If price is skyrocketing (high total vol, low downside vol), we maintain FULL size.
    3. If price is crashing (high downside vol), we cut size aggressivey.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        
    def _calc_downside_deviation(self, returns, window=30):
        # Filter for negative returns only, replace positive with 0
        downside_rets = returns.copy()
        downside_rets[downside_rets > 0] = 0
        # Calculate rolling standard deviation of these negative returns
        return downside_rets.rolling(window=window).std() * np.sqrt(252)

    def generate_signals(self):
        if self.data is None or self.data.empty: return
        df = self.data.copy()
        
        # 1. Base Features
        df['FracDiff'] = FeatureLab.frac_diff_fixed(df['Adj Close'].apply(np.log), d=0.4, window=50)
        df['RSI'] = FeatureLab.compute_rsi(df['Adj Close'])
        df['SMA_200'] = df['Adj Close'].rolling(200).mean()
        
        # 2. Base Signal (Trend Following)
        # We want to be long if FracDiff is positive OR Price > SMA200 (Structural Bull)
        df['Signal'] = 0.0
        
        bull_condition = (df['FracDiff'] > 0) | (df['Adj Close'] > df['SMA_200'])
        df.loc[bull_condition, 'Signal'] = 1.0
        
        # 3. Sortino-Based Sizing (The Alpha Fix)
        # Calculate Downside Volatility
        df['Downside_Vol'] = self._calc_downside_deviation(df['Returns'])
        
        # Safety check for division by zero
        df['Downside_Vol'].replace(0, 0.001, inplace=True)
        
        # Target Volatility (e.g., 10% downside risk, which is generous)
        target_downside = 0.10 
        
        # Scalar = Target / Downside. 
        # If Downside Vol is 0 (Pure uptrend), Scalar explodes. We clip at 2.0 (2x leverage).
        # This allows the strategy to ride NVDA up without cutting exposure.
        df['Vol_Scaler'] = (target_downside / df['Downside_Vol']).clip(upper=2.0)
        
        # 4. Crisis Management (VIX Proxy or Price Action Veto)
        # If RSI < 30 (Oversold) AND Downside Vol is spiking, we cut leverage
        panic_mode = (df['RSI'] < 30) & (df['Downside_Vol'] > 0.20)
        df.loc[panic_mode, 'Vol_Scaler'] = 0.5
        
        df['Signal'] = df['Signal'] * df['Vol_Scaler']
        
        self.data = df

### V23_TurboTrend

In [155]:
class StrategyV23_TurboTrend(BaseStrategy):
    """
    V23: The 'Turbo Trend' Strategy.
    
    Designed specifically to capture 'Super-Trends' (NVDA, Crypto, XLE) where
    volatility targeting (V3, V22) fails by reducing size too early.
    
    MECHANISM:
    1. Entry: Structural Trend (Price > EMA_Short > EMA_Long).
    2. Sizing: Fixed High Conviction (1.0 or 1.3). NO volatility dampening.
    3. Exit: Chandelier Exit (ATR Trailing Stop).
       - We trail the stop loss behind the highest high.
       - If price hits the stop, we exit immediately.
       
    This ensures we stay 100% invested during the parabolic phase and only 
    exit when the structural trend actually breaks.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        
    def _calculate_atr(self, df, window=14):
        high_low = df['High'] - df['Low']
        high_close = np.abs(df['High'] - df['Close'].shift())
        low_close = np.abs(df['Low'] - df['Close'].shift())
        ranges = pd.concat([high_low, high_close, low_close], axis=1)
        true_range = ranges.max(axis=1)
        return true_range.rolling(window).mean()

    def generate_signals(self):
        if self.data is None or self.data.empty: return
        df = self.data.copy()
        
        # 1. Trend Features
        df['EMA_20'] = df['Adj Close'].ewm(span=20, adjust=False).mean()
        df['EMA_50'] = df['Adj Close'].ewm(span=50, adjust=False).mean()
        df['EMA_200'] = df['Adj Close'].ewm(span=200, adjust=False).mean()
        
        # 2. ATR for Trailing Stop
        df['ATR'] = self._calculate_atr(df, window=21)
        
        # 3. Chandelier Exit Calculation
        # The "Floor" moves up, but never down, while in a trade.
        # Here we simulate the logic vectorially.
        
        rolling_max = df['Adj Close'].rolling(window=22).max()
        # A tighter multiplier (3.0) allows room for volatility but exits on crash
        df['Trailing_Stop'] = rolling_max - (3.0 * df['ATR'])
        
        # 4. Signal Generation
        df['Signal'] = 0.0
        
        # ENTRY CONDITION:
        # A. Golden Alignment: Price > EMA20 > EMA50
        # B. Structural Bull: Price > EMA200 (Filter out dead cat bounces in bear markets)
        entry_cond = (df['Adj Close'] > df['EMA_20']) & \
                     (df['EMA_20'] > df['EMA_50']) & \
                     (df['Adj Close'] > df['EMA_200'])
        
        # EXIT CONDITION:
        # Price closes below Trailing Stop
        exit_cond = (df['Adj Close'] < df['Trailing_Stop'])
        
        # Vectorized Position Management
        # We start with 0. If Entry -> 1. If Exit -> 0.
        # We need to forward-fill the "In Trade" state.
        
        # Create a state mask
        signals = np.zeros(len(df))
        in_trade = False
        
        # Iterating for state management (cleanest way for trailing stops)
        price = df['Adj Close'].values
        ema_20 = df['EMA_20'].values
        ema_50 = df['EMA_50'].values
        ema_200 = df['EMA_200'].values
        stops = df['Trailing_Stop'].values
        
        for i in range(1, len(df)):
            # Update State
            if in_trade:
                # Check Exit
                if price[i] < stops[i]:
                    signals[i] = 0
                    in_trade = False
                else:
                    signals[i] = 1 # Hold
            else:
                # Check Entry
                if (price[i] > ema_20[i]) and (ema_20[i] > ema_50[i]) and (price[i] > ema_200[i]):
                    signals[i] = 1
                    in_trade = True
                else:
                    signals[i] = 0
                    
        df['Signal'] = signals
        
        # 5. NO Volatility Targeting
        # We apply a fixed leverage for strong trends if desired, or just 1.0.
        # Let's set to 1.0 (Full Invested). 
        # Note: If you want leverage on NVDA, change this to 1.3
        
        self.data = df

### V24_Accelerating

In [156]:
class StrategyV24_Accelerator(BaseStrategy):
    """
    V24: The 'Accelerator' Strategy.
    
    Evolution of V23 (Turbo). Designed for Tech Momentum (NVDA/TSLA).
    
    KEY FEATURES:
    1. Structural Entry: Price > EMA20 > EMA50 > EMA200.
    2. Accelerator Exit: A Dynamic Chandelier Exit.
       - Base Stop: 3.5x ATR (Loose, allows initial volatility).
       - Level 1: If Profit > 15%, tighten to 2.5x ATR.
       - Level 2: If Profit > 30%, tighten to 1.5x ATR.
       
    This solves the "TSLA Problem" where massive gains evaporate 
    because the trailing stop was too loose at the top.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        
    def _calculate_atr(self, df, window=14):
        high_low = df['High'] - df['Low']
        high_close = np.abs(df['High'] - df['Close'].shift())
        low_close = np.abs(df['Low'] - df['Close'].shift())
        ranges = pd.concat([high_low, high_close, low_close], axis=1)
        true_range = ranges.max(axis=1)
        return true_range.rolling(window).mean()

    def generate_signals(self):
        if self.data is None or self.data.empty: return
        df = self.data.copy()
        
        # 1. Technicals
        df['EMA_20'] = df['Adj Close'].ewm(span=20, adjust=False).mean()
        df['EMA_50'] = df['Adj Close'].ewm(span=50, adjust=False).mean()
        df['EMA_200'] = df['Adj Close'].ewm(span=200, adjust=False).mean()
        df['ATR'] = self._calculate_atr(df, window=14)
        
        # 2. Vectorized Backtest Loop (Required for Dynamic Stops)
        # We cannot do this purely vectorially because 'Profit' depends on 'Entry Price'
        
        signals = np.zeros(len(df))
        price = df['Adj Close'].values
        high = df['High'].values
        ema_20 = df['EMA_20'].values
        ema_50 = df['EMA_50'].values
        ema_200 = df['EMA_200'].values
        atr = df['ATR'].values
        
        in_trade = False
        entry_price = 0.0
        highest_high_since_entry = 0.0
        
        for i in range(1, len(df)):
            if np.isnan(atr[i]): continue
            
            if in_trade:
                # A. Update Highest High
                if high[i] > highest_high_since_entry:
                    highest_high_since_entry = high[i]
                
                # B. Calculate Dynamic Multiplier based on Unrealized ROI (Peak)
                roi = (highest_high_since_entry - entry_price) / entry_price
                
                if roi > 0.30:
                    mult = 1.5 # Lock it in tight!
                elif roi > 0.15:
                    mult = 2.5 # Tighten up
                else:
                    mult = 3.5 # Give it room
                
                # C. Calculate Stop Price
                stop_price = highest_high_since_entry - (mult * atr[i])
                
                # D. Check Exit
                if price[i] < stop_price:
                    signals[i] = 0
                    in_trade = False
                else:
                    signals[i] = 1 # Stay Long
            
            else:
                # Check Entry: Strict Golden Alignment
                is_bull_trend = (price[i] > ema_20[i]) and \
                                (ema_20[i] > ema_50[i]) and \
                                (ema_50[i] > ema_200[i])
                
                if is_bull_trend:
                    signals[i] = 1
                    in_trade = True
                    entry_price = price[i]
                    highest_high_since_entry = high[i]
                else:
                    signals[i] = 0
        
        df['Signal'] = signals
        self.data = df

### V25_Precision

In [157]:
class StrategyV25_PrecisionTrend(BaseStrategy):
    """
    V25: The 'Precision Trend' Strategy.
    
    Replaces EMA Crossovers (laggy) with RSI Regime logic (fast).
    Designed to function as the "Bull Leg" of Agent V4.
    
    MECHANISM:
    1. Filter: Structural Bull (Price > SMA200).
    2. Entry: Momentum Ignition (RSI crosses above 50).
    3. Exit: Chandelier Exit (Highest High - 3.0 * ATR).
    
    Why this beats V24:
    - RSI > 50 gets us into JPM/NVDA moves weeks earlier than EMA20 > EMA50.
    - Standard Chandelier Stop prevents the "suffocation" seen in TSLA V24.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        
    def _calculate_atr(self, df, window=14):
        high_low = df['High'] - df['Low']
        high_close = np.abs(df['High'] - df['Close'].shift())
        low_close = np.abs(df['Low'] - df['Close'].shift())
        ranges = pd.concat([high_low, high_close, low_close], axis=1)
        true_range = ranges.max(axis=1)
        return true_range.rolling(window).mean()

    def generate_signals(self):
        if self.data is None or self.data.empty: return
        df = self.data.copy()
        
        # 1. Features
        df['SMA_200'] = df['Adj Close'].rolling(200).mean()
        df['RSI'] = FeatureLab.compute_rsi(df['Adj Close'], window=14)
        df['ATR'] = self._calculate_atr(df, window=14)
        
        # 2. Chandelier Exit (Vectorized Simulation)
        # We simulate the trailing stop logic.
        
        signals = np.zeros(len(df))
        price = df['Adj Close'].values
        high = df['High'].values
        sma_200 = df['SMA_200'].values
        rsi = df['RSI'].values
        atr = df['ATR'].values
        
        in_trade = False
        highest_high = 0.0
        
        for i in range(1, len(df)):
            if np.isnan(sma_200[i]) or np.isnan(atr[i]): continue
            
            if in_trade:
                # Update Trailing Stop
                if high[i] > highest_high:
                    highest_high = high[i]
                
                # Standard Chandelier: 3.0 ATR
                stop_price = highest_high - (3.0 * atr[i])
                
                # Exit Trigger
                if price[i] < stop_price:
                    signals[i] = 0
                    in_trade = False
                else:
                    signals[i] = 1
            else:
                # Entry Trigger
                # 1. Structural Bull (Price > SMA200)
                # 2. Momentum Active (RSI > 50) - "The Bull Zone"
                if (price[i] > sma_200[i]) and (rsi[i] > 50):
                    signals[i] = 1
                    in_trade = True
                    highest_high = high[i]
                else:
                    signals[i] = 0
                    
        df['Signal'] = signals
        self.data = df

### Agent V1: Auto Regime

In [158]:
class AutoRegimeAgent(BaseStrategy):
    """
    The 'Meta-Controller' Agent.
    
    Instead of blending strategies (Ensemble), this Agent acts as a Switch.
    It identifies the market 'Context' and deploys the single best specialist 
    model for that context.
    
    STATES:
    1. TRENDING (ADX > 25): Deploy V22_Sortino (Ride the wave).
    2. MEAN REVERSION (ADX < 20): Deploy V9_Unshackled (Trade the chop).
    3. STRESS (VIX > 25 or TNX Spiking): Deploy Cash/Safety.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        # Initialize the specialists
        self.trend_model = StrategyV22_Sortino_Unbound(ticker, start_date, end_date)
        self.chop_model = StrategyV9_RegimeUnshackled(ticker, start_date, end_date)
        self.macro_data = None

    def fetch_data(self, warmup_years=2):
        # Fetch underlying data once
        self.trend_model.fetch_data(warmup_years)
        self.chop_model.fetch_data(warmup_years)
        
        if self.trend_model.data is not None:
            self.data = self.trend_model.data.copy()
            
        # Fetch Context Data (VIX)
        try:
            vix = yf.download("^VIX", start=self.data.index[0], end=self.end_date, progress=False, auto_adjust=True)
            if isinstance(vix.columns, pd.MultiIndex): vix.columns = vix.columns.get_level_values(0)
            self.macro_data = vix['Close'].reindex(self.data.index).fillna(method='ffill')
        except:
            self.macro_data = pd.Series(20, index=self.data.index)

    def _calculate_adx(self, df, window=14):
        """Calculate Trend Strength (ADX)"""
        high = df['High']
        low = df['Low']
        close = df['Close']
        
        plus_dm = high.diff()
        minus_dm = low.diff()
        plus_dm[plus_dm < 0] = 0
        minus_dm[minus_dm > 0] = 0
        
        tr1 = pd.DataFrame(high - low)
        tr2 = pd.DataFrame(abs(high - close.shift(1)))
        tr3 = pd.DataFrame(abs(low - close.shift(1)))
        frames = [tr1, tr2, tr3]
        tr = pd.concat(frames, axis=1, join='outer').max(axis=1)
        atr = tr.rolling(window).mean()
        
        plus_di = 100 * (plus_dm.ewm(alpha=1/window).mean() / atr)
        minus_di = 100 * (abs(minus_dm).ewm(alpha=1/window).mean() / atr)
        dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
        adx = dx.rolling(window).mean()
        return adx.fillna(20)

    def generate_signals(self):
        if self.data is None: return
        
        # 1. Generate Specialist Signals
        self.trend_model.generate_signals()
        self.chop_model.generate_signals()
        
        # 2. Align Data
        df = self.data.copy()
        df['Sig_Trend'] = self.trend_model.data['Signal']
        df['Sig_Chop'] = self.chop_model.data['Signal']
        df['VIX'] = self.macro_data
        
        # 3. Calculate Context (ADX)
        df['ADX'] = self._calculate_adx(df)
        
        # 4. Agent Logic (The Switchboard)
        df['Active_Model'] = 'CASH'
        df['Signal'] = 0.0
        
        # Logic Loop
        # If VIX is high, we don't trust the Trend model (too much risk), 
        # but we might trust V9 (which has Bear/Panic logic).
        
        # Vectorized implementation for speed
        # Condition A: Strong Trend -> Use V22
        cond_trend = (df['ADX'] > 25) & (df['VIX'] < 30)
        
        # Condition B: Chop/Noise -> Use V9
        cond_chop = (df['ADX'] <= 25) & (df['VIX'] < 35)
        
        # Condition C: Extreme Stress -> Cash (Implicit in Signal=0 default)
        
        # Apply Logic
        # We prefer Trend if both are true (Trend is usually more profitable)
        df.loc[cond_chop, 'Signal'] = df.loc[cond_chop, 'Sig_Chop']
        df.loc[cond_trend, 'Signal'] = df.loc[cond_trend, 'Sig_Trend']
        
        # Final Volatility Safety Overlay (Agent Level)
        # Even if V22 says "Go", if VIX > 40, the Agent vetoes.
        df.loc[df['VIX'] > 40, 'Signal'] = 0.0
        
        self.data = df

### Agent V2

In [159]:
import pandas as pd
import numpy as np
import yfinance as yf
# NOTE: Assumes BaseStrategy, StrategyV9_RegimeUnshackled, and StrategyV23_TurboTrend are available

class AutoRegimeAgent_V2(BaseStrategy):
    """
    Agent V2: Macro-Aware & Turbo-Enabled.
    
    Improves upon the original Agent by:
    1. Using V23 (TurboTrend) for Growth phases (Solving NVDA/XLE).
    2. Incorporating Macro Data (VIX/TNX) to handle Cyclicals (Solving JPM).
    3. Retaining V9 (Unshackled) for Chop/Correction phases (Solving TSLA/BABA).
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        # Specialist 1: The Trend Hunter
        self.trend_model = StrategyV23_TurboTrend(ticker, start_date, end_date)
        # Specialist 2: The Chop Survivor
        self.chop_model = StrategyV9_RegimeUnshackled(ticker, start_date, end_date)
        
        self.macro_data = None

    def fetch_data(self, warmup_years=2):
        # 1. Fetch Specialist Data
        self.trend_model.fetch_data(warmup_years)
        self.chop_model.fetch_data(warmup_years)
        
        if self.trend_model.data is not None:
            self.data = self.trend_model.data.copy()
            
        # 2. Fetch Macro Context (VIX and TNX)
        try:
            start_dt = self.data.index[0]
            macro_tickers = ["^VIX", "^TNX"]
            macro_df = yf.download(macro_tickers, start=start_dt, end=self.end_date, progress=False, auto_adjust=True)
            
            # Handle MultiIndex
            if isinstance(macro_df.columns, pd.MultiIndex):
                self.macro_data = macro_df['Close'].fillna(method='ffill')
            else:
                self.macro_data = macro_df['Close'].fillna(method='ffill')
                
            # Align to asset index
            self.macro_data = self.macro_data.reindex(self.data.index).fillna(method='ffill')
            
        except Exception as e:
            print(f"Agent Macro Fetch Error: {e}")
            # Fallback
            self.macro_data = pd.DataFrame({'^VIX': 20, '^TNX': 4.0}, index=self.data.index)

    def _calculate_adx(self, df, window=14):
        # Standard ADX calculation
        high = df['High']
        low = df['Low']
        close = df['Close']
        
        plus_dm = high.diff()
        minus_dm = low.diff()
        plus_dm[plus_dm < 0] = 0
        minus_dm[minus_dm > 0] = 0
        
        tr1 = pd.DataFrame(high - low)
        tr2 = pd.DataFrame(abs(high - close.shift(1)))
        tr3 = pd.DataFrame(abs(low - close.shift(1)))
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = tr.rolling(window).mean()
        
        plus_di = 100 * (plus_dm.ewm(alpha=1/window).mean() / atr)
        minus_di = 100 * (abs(minus_dm).ewm(alpha=1/window).mean() / atr)
        dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
        adx = dx.rolling(window).mean().fillna(20)
        return adx

    def generate_signals(self):
        if self.data is None or self.macro_data is None: return
        
        # 1. Run Specialists
        self.trend_model.generate_signals()
        self.chop_model.generate_signals()
        
        # 2. Build Decision Matrix
        df = self.data.copy()
        df['Sig_Turbo'] = self.trend_model.data['Signal']
        df['Sig_Chop'] = self.chop_model.data['Signal']
        
        # Context Features
        df['ADX'] = self._calculate_adx(df)
        df['VIX'] = self.macro_data['^VIX']
        
        # TNX Rate of Change (for Macro Stress)
        df['TNX_ROC'] = self.macro_data['^TNX'].pct_change(20)
        
        # 3. Agent State Logic
        df['Signal'] = 0.0
        
        # Thresholds
        VIX_PANIC = 30.0
        TNX_SHOCK = 0.15 # 15% rise in yields in a month
        ADX_TREND = 25.0
        
        # State A: MACRO STRESS (The "JPM/Real Estate" Logic)
        # If yields are spiking or fear is extreme, we default to safety (Cash or V9 Defensive).
        # However, if ADX is EXTREME (>50), we might ignore Macro (Idiosyncratic Breakout like Crypto).
        is_macro_stress = (df['VIX'] > VIX_PANIC) | (df['TNX_ROC'] > TNX_SHOCK)
        
        # State B: TURBO TREND (The "NVDA" Logic)
        # Strong ADX and Price Structure.
        is_turbo_trend = (df['ADX'] > ADX_TREND)
        
        # LOGIC TREE
        # Default: Use Chop Model (V9) - Good for accumulation and mean reversion
        df['Signal'] = df['Sig_Chop']
        
        # Override 1: If Turbo Trend -> Switch to V23
        # We allow V23 to override Macro Stress IF the trend is extremely strong (ADX > 40)
        # This solves XLE/NVDA rallying despite market fear.
        strong_override = (df['ADX'] > 40)
        
        use_turbo = (is_turbo_trend & ~is_macro_stress) | strong_override
        df.loc[use_turbo, 'Signal'] = df.loc[use_turbo, 'Sig_Turbo']
        
        # Override 2: If Macro Stress (and no override) -> Safety
        # In this case, we can either go to 0.0 (Cash) or use V9's defensive logic.
        # V9 already has logic for Bears, but let's be safer.
        force_cash = is_macro_stress & ~strong_override
        df.loc[force_cash, 'Signal'] = 0.0
        
        self.data = df

### Agent V3

In [160]:
class AutoRegimeAgent_V3(BaseStrategy):
    """
    Agent V3: The 'Smart' Tech Agent.
    
    Fixes the 'Blind ADX' bug from V2.
    
    LOGIC:
    1. Panic Filter: If VIX > 32, Cash. (Ignores TNX to save Banks).
    2. Trend Filter: 
       - If ADX > 25 AND PDI > MDI (Bull Trend) -> V24 Accelerator.
       - If ADX > 25 AND PDI < MDI (Bear Trend) -> V9 Unshackled (Mean Rev).
    3. Chop Filter:
       - If ADX < 25 -> V9 Unshackled.
       
    This ensures we don't apply Turbo strategies to crashing stocks (BABA),
    restoring the defensive Alpha.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        # Specialist 1: The Accelerator (Bull Trend)
        self.trend_model = StrategyV24_Accelerator(ticker, start_date, end_date)
        # Specialist 2: The Unshackled (Bear/Chop)
        self.chop_model = StrategyV9_RegimeUnshackled(ticker, start_date, end_date)
        
        self.macro_data = None

    def fetch_data(self, warmup_years=2):
        self.trend_model.fetch_data(warmup_years)
        self.chop_model.fetch_data(warmup_years)
        
        if self.trend_model.data is not None:
            self.data = self.trend_model.data.copy()
            
        try:
            # Only fetch VIX for this version
            vix = yf.download("^VIX", start=self.data.index[0], end=self.end_date, progress=False, auto_adjust=True)
            if isinstance(vix.columns, pd.MultiIndex): vix.columns = vix.columns.get_level_values(0)
            self.macro_data = vix['Close'].reindex(self.data.index).fillna(method='ffill')
        except:
            self.macro_data = pd.Series(20, index=self.data.index)

    def _calculate_dmi(self, df, window=14):
        """Calculates ADX, Plus_DI, and Minus_DI"""
        high = df['High']
        low = df['Low']
        close = df['Close']
        
        plus_dm = high.diff()
        minus_dm = low.diff()
        plus_dm[plus_dm < 0] = 0
        minus_dm[minus_dm > 0] = 0
        
        tr1 = pd.DataFrame(high - low)
        tr2 = pd.DataFrame(abs(high - close.shift(1)))
        tr3 = pd.DataFrame(abs(low - close.shift(1)))
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = tr.rolling(window).mean()
        
        # Calculate DI+ and DI-
        plus_di = 100 * (plus_dm.ewm(alpha=1/window).mean() / atr)
        minus_di = 100 * (abs(minus_dm).ewm(alpha=1/window).mean() / atr)
        
        dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
        adx = dx.rolling(window).mean().fillna(20)
        
        return adx, plus_di, minus_di

    def generate_signals(self):
        if self.data is None or self.macro_data is None: return
        
        # 1. Run Specialists
        self.trend_model.generate_signals()
        self.chop_model.generate_signals()
        
        # 2. Build Decision Context
        df = self.data.copy()
        df['Sig_Turbo'] = self.trend_model.data['Signal']
        df['Sig_Chop'] = self.chop_model.data['Signal']
        df['VIX'] = self.macro_data
        
        # Calculate Directional Indicators
        df['ADX'], df['PDI'], df['MDI'] = self._calculate_dmi(df)
        
        # 3. Agent Logic
        df['Signal'] = 0.0
        
        # A. PANIC (Global Circuit Breaker)
        is_panic = (df['VIX'] > 32)
        
        # B. BULL TREND (Strong ADX + Bulls in Control)
        # Note: We rely on PDI > MDI to confirm direction
        is_bull_trend = (df['ADX'] > 25) & (df['PDI'] > df['MDI'])
        
        # C. Logic Application
        
        # Default: Use Chop Model (Handles Chop AND Bear Trends via Mean Rev)
        # This covers BABA (Bear) and TSLA (Chop phases)
        df['Signal'] = df['Sig_Chop']
        
        # Override: Use Turbo Model ONLY if Bull Trend AND Not Panic
        # This covers NVDA/JPM Rallies
        use_turbo = (is_bull_trend & ~is_panic)
        df.loc[use_turbo, 'Signal'] = df.loc[use_turbo, 'Sig_Turbo']
        
        # Override: Force Cash if Panic (Optional, V9 handles this partly, but let's be safe)
        df.loc[is_panic, 'Signal'] = 0.0
        
        self.data = df

### Agent V4

In [161]:
class AutoRegimeAgent_V4(BaseStrategy):
    """
    Agent V4: The 'Guardrail' Agent.
    
    The definitive solution for the Tech/Cyclical split.
    
    CORE LOGIC:
    The SMA200 is the "Great Divider".
    - Below SMA200 (Bear Market): Trend Following FAILS (Whipsaw). We MUST use Mean Reversion (V9).
    - Above SMA200 (Bull Market): Mean Reversion UNDERPERFORMS. We MUST use Trend Following (V25).
    
    This simple rule solves the BABA/TSLA crash performance while preserving NVDA upside.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        # The Bull Specialist
        self.bull_model = StrategyV25_PrecisionTrend(ticker, start_date, end_date)
        # The Bear Specialist
        self.bear_model = StrategyV9_RegimeUnshackled(ticker, start_date, end_date)
        
        self.macro_data = None

    def fetch_data(self, warmup_years=2):
        self.bull_model.fetch_data(warmup_years)
        self.bear_model.fetch_data(warmup_years)
        
        if self.bull_model.data is not None:
            self.data = self.bull_model.data.copy()
            
        try:
            vix = yf.download("^VIX", start=self.data.index[0], end=self.end_date, progress=False, auto_adjust=True)
            if isinstance(vix.columns, pd.MultiIndex): vix.columns = vix.columns.get_level_values(0)
            self.macro_data = vix['Close'].reindex(self.data.index).fillna(method='ffill')
        except:
            self.macro_data = pd.Series(20, index=self.data.index)

    def generate_signals(self):
        if self.data is None or self.macro_data is None: return
        
        # 1. Run Specialists
        self.bull_model.generate_signals()
        self.bear_model.generate_signals()
        
        # 2. Context
        df = self.data.copy()
        df['Sig_Bull'] = self.bull_model.data['Signal']
        df['Sig_Bear'] = self.bear_model.data['Signal']
        df['VIX'] = self.macro_data
        
        # 3. Structural Filter (The Guardrail)
        df['SMA_200'] = df['Adj Close'].rolling(200).mean()
        
        # 4. Logic Tree
        df['Signal'] = 0.0
        
        # Condition A: Structural Bear (Price < SMA200) OR Panic (VIX > 32)
        # In this zone, we ONLY trust the Unshackled V9 model (which buys deep dips and shorts rips)
        is_defensive_zone = (df['Adj Close'] < df['SMA_200']) | (df['VIX'] > 32)
        
        # Condition B: Structural Bull (Price > SMA200) AND Calm (VIX < 32)
        # In this zone, we trust the Precision Trend V25 model
        is_aggressive_zone = ~is_defensive_zone
        
        # Apply Signals
        # Note: Vectorized assignment
        df.loc[is_defensive_zone, 'Signal'] = df.loc[is_defensive_zone, 'Sig_Bear']
        df.loc[is_aggressive_zone, 'Signal'] = df.loc[is_aggressive_zone, 'Sig_Bull']
        
        self.data = df

### Agent V5

In [162]:
class AutoRegimeAgent_V5(BaseStrategy):
    """
    Agent V5: The Composite Agent.
    
    Combines the best-performing components from previous iterations:
    1. BULL ENGINE: V23_TurboTrend (From Agent V2) - Best for NVDA/TSLA.
    2. BEAR ENGINE: V9_Unshackled (From Agent V1) - Best for BABA/Chop.
    3. LOGIC: V4 Guardrails + V12 Macro Awareness.
    
    HIERARCHY:
    1. Structural Bear (Price < SMA200) -> Force V9.
    2. Macro Stress (High VIX/Rates)   -> Force V9.
    3. Bull Trend (ADX > 20)           -> Force V23.
    4. Chop (Low ADX)                  -> Force V9.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        # Specialist 1: Turbo Trend (Restored)
        self.bull_model = StrategyV23_TurboTrend(ticker, start_date, end_date)
        # Specialist 2: Unshackled Regime
        self.bear_model = StrategyV9_RegimeUnshackled(ticker, start_date, end_date)
        
        self.macro_data = None

    def fetch_data(self, warmup_years=2):
        self.bull_model.fetch_data(warmup_years)
        self.bear_model.fetch_data(warmup_years)
        
        if self.bull_model.data is not None:
            self.data = self.bull_model.data.copy()
            
        try:
            # Fetch Macro Data (V12 Logic)
            tickers = ["^VIX", "^TNX"]
            macro_df = yf.download(tickers, start=self.data.index[0], end=self.end_date, progress=False, auto_adjust=True)
            if isinstance(macro_df.columns, pd.MultiIndex): 
                # Flatten
                self.macro_data = pd.DataFrame({
                    'VIX': macro_df['Close']['^VIX'],
                    'TNX': macro_df['Close']['^TNX']
                })
            else:
                 self.macro_data = pd.DataFrame({
                    'VIX': macro_df['Close']['^VIX'],
                    'TNX': macro_df['Close']['^TNX']
                })
            
            # Reindex to match asset
            self.macro_data = self.macro_data.reindex(self.data.index).fillna(method='ffill')
            
        except Exception as e:
            print(f"Agent V5 Macro Fetch Error: {e}")
            self.macro_data = pd.DataFrame({'VIX': 20, 'TNX': 4.0}, index=self.data.index)

    def _calculate_adx(self, df, window=14):
        high = df['High']
        low = df['Low']
        close = df['Close']
        
        plus_dm = high.diff()
        minus_dm = low.diff()
        plus_dm[plus_dm < 0] = 0
        minus_dm[minus_dm > 0] = 0
        
        tr1 = pd.DataFrame(high - low)
        tr2 = pd.DataFrame(abs(high - close.shift(1)))
        tr3 = pd.DataFrame(abs(low - close.shift(1)))
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = tr.rolling(window).mean()
        
        plus_di = 100 * (plus_dm.ewm(alpha=1/window).mean() / atr)
        minus_di = 100 * (abs(minus_dm).ewm(alpha=1/window).mean() / atr)
        dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
        adx = dx.rolling(window).mean().fillna(20)
        return adx

    def generate_signals(self):
        if self.data is None or self.macro_data is None: return
        
        # 1. Run Specialists
        self.bull_model.generate_signals()
        self.bear_model.generate_signals()
        
        # 2. Construct Context
        df = self.data.copy()
        df['Sig_Bull'] = self.bull_model.data['Signal']
        df['Sig_Bear'] = self.bear_model.data['Signal']
        
        # Features
        df['SMA_200'] = df['Adj Close'].rolling(200).mean()
        df['ADX'] = self._calculate_adx(df)
        
        # Macro Stress (V12 Logic)
        macro = self.macro_data.copy()
        macro['TNX_ROC'] = macro['TNX'].pct_change(20)
        
        # Stress Conditions
        # VIX > 30 is Panic
        # TNX Rising > 15% in a month is Rate Shock (Bad for JPM/Tech)
        df['Macro_Panic'] = (macro['VIX'] > 30) | (macro['TNX_ROC'] > 0.15)
        
        # 3. Agent Logic Hierarchy
        df['Signal'] = 0.0
        
        # Check 1: Structural Bear Market (Iron Floor)
        # If Price < SMA200, we DO NOT TRUST TREND MODELS. We use V9.
        is_structural_bear = df['Adj Close'] < df['SMA_200']
        
        # Check 2: Macro Panic
        # If VIX/Rates are spiking, we go Defensive (V9)
        is_panic = df['Macro_Panic']
        
        # Check 3: Bull Trend
        # If NOT Bear AND NOT Panic AND ADX > 20
        is_bull_trend = (~is_structural_bear) & (~is_panic) & (df['ADX'] > 20)
        
        # Default: V9 (Chop/Defensive)
        df['Signal'] = df['Sig_Bear']
        
        # Override: V23 (Turbo)
        # Only if conditions are perfect
        df.loc[is_bull_trend, 'Signal'] = df.loc[is_bull_trend, 'Sig_Bull']
        
        # Note: If is_structural_bear or is_panic is True, we stay with the Default (V9).
        # This fixes BABA (Bear) and JPM (Panic).
        
        self.data = df

### Agent V6: Darwin

In [163]:
class AutoRegimeAgent_V6_Darwin(BaseStrategy):
    """
    Agent V6: The 'Darwinian' Adaptive Agent.
    
    Instead of hardcoded logic rules (IF/ELSE), this Agent uses 
    Walk-Forward Optimization (Online Learning) to select the best model.
    
    MECHANISM:
    1. Runs V9, V12, and V23 in the background (Virtual Accounts).
    2. Calculates a Rolling Sharpe Ratio (63-day lookback) for each.
    3. Allocates 100% capital to the strategy with the highest Realized Sharpe.
    4. "Cash Veto": If the best strategy has Negative Sharpe, go to Cash.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        # The Candidates
        self.strat_v9 = StrategyV9_RegimeUnshackled(ticker, start_date, end_date)
        self.strat_v12 = StrategyV12_Macro_Switch(ticker, start_date, end_date)
        self.strat_v23 = StrategyV23_TurboTrend(ticker, start_date, end_date)
        
        # Parameters
        self.lookback_window = 63  # 3 Months for Regime Detection
        
    def fetch_data(self, warmup_years=2):
        # Fetch data for all candidates
        self.strat_v9.fetch_data(warmup_years)
        self.strat_v12.fetch_data(warmup_years)
        self.strat_v23.fetch_data(warmup_years)
        
        # Use V23 data as the base (it likely has the cleanest index)
        if self.strat_v23.data is not None:
            self.data = self.strat_v23.data.copy()
            
    def generate_signals(self):
        # Check if sub-strategies have data
        if self.strat_v9.data is None or self.strat_v12.data is None or self.strat_v23.data is None:
            return

        # 1. Generate Sub-Strategy Signals
        self.strat_v9.generate_signals()
        self.strat_v12.generate_signals()
        self.strat_v23.generate_signals()
        
        # 2. Prepare Virtual Account DataFrame
        df = self.data.copy()
        
        # Align signals to the main index (Left Join)
        s9 = self.strat_v9.data['Signal'].reindex(df.index).fillna(0)
        s12 = self.strat_v12.data['Signal'].reindex(df.index).fillna(0)
        s23 = self.strat_v23.data['Signal'].reindex(df.index).fillna(0)
        
        # Calculate Daily Returns of the Asset
        # (Assuming 'Returns' column exists, if not calculate it)
        if 'Returns' not in df.columns:
            df['Returns'] = df['Adj Close'].pct_change()
            
        market_ret = df['Returns'].fillna(0)
        
        # 3. Calculate Virtual Strategy Returns (Realized Performance)
        # Shift signal by 1 because Signal T acts on Return T+1
        # (Or Signal T is generated at Close T, acts on Close T+1)
        # Standard Backtest convention: Ret_Strat = Signal.shift(1) * Returns
        
        ret_v9 = s9.shift(1) * market_ret
        ret_v12 = s12.shift(1) * market_ret
        ret_v23 = s23.shift(1) * market_ret
        
        # 4. Walk-Forward Selection Loop (Vectorized Rolling)
        
        # Calculate Rolling Mean and Std for Sharpe Ratio
        # Epsilon to avoid div by zero
        eps = 1e-9
        
        # V9 Metrics
        roll_mean_9 = ret_v9.rolling(self.lookback_window).mean()
        roll_std_9 = ret_v9.rolling(self.lookback_window).std() + eps
        sharpe_9 = (roll_mean_9 / roll_std_9) * np.sqrt(252)
        
        # V12 Metrics
        roll_mean_12 = ret_v12.rolling(self.lookback_window).mean()
        roll_std_12 = ret_v12.rolling(self.lookback_window).std() + eps
        sharpe_12 = (roll_mean_12 / roll_std_12) * np.sqrt(252)
        
        # V23 Metrics
        roll_mean_23 = ret_v23.rolling(self.lookback_window).mean()
        roll_std_23 = ret_v23.rolling(self.lookback_window).std() + eps
        sharpe_23 = (roll_mean_23 / roll_std_23) * np.sqrt(252)
        
        # 5. Selection Logic
        
        # Stack scores into a DataFrame for easy comparison
        scores = pd.DataFrame({
            'V9': sharpe_9,
            'V12': sharpe_12,
            'V23': sharpe_23
        })
        
        # Identify Winner (Column name with max value)
        # idxmax returns the column name of the max value
        winners = scores.idxmax(axis=1)
        best_scores = scores.max(axis=1)
        
        # 6. Construct Final Signal
        # We must decide TODAY'S signal based on YESTERDAY'S winner to avoid lookahead.
        # However, the Rolling calculation already includes today's return in the window end.
        # To be strictly safe: We use the Winner determined at T-1 to choose the Signal from T.
        
        # Shift the decision by 1 day
        active_strategy = winners.shift(1)
        active_score = best_scores.shift(1)
        
        final_signal = np.zeros(len(df))
        
        # Map strategy names to their signal arrays
        sig_map = {
            'V9': s9.values,
            'V12': s12.values,
            'V23': s23.values
        }
        
        strat_names = active_strategy.values
        scores_vals = active_score.values
        
        for i in range(len(df)):
            strat = strat_names[i]
            score = scores_vals[i]
            
            # Cash Veto: If the best strategy is losing money (Sharpe < 0), go to Cash.
            if score < 0 or pd.isna(score):
                final_signal[i] = 0.0
            elif isinstance(strat, str) and strat in sig_map:
                final_signal[i] = sig_map[strat][i]
            else:
                final_signal[i] = 0.0
                
        df['Signal'] = final_signal
        
        # Store for debugging/visualization
        df['Active_Strat'] = active_strategy
        df['Active_Score'] = active_score
        
        self.data = df

# Execution

In [164]:
class RobustBenchmark:
    """
    Implements Walk-Forward Analysis and Deflated Sharpe Ratio logic.
    Benchmarks multiple strategies without look-ahead bias[cite: 275].
    """
    def __init__(self, tickers, start_date, end_date):
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.results = []

    def run(self):
        print(f"{'STRATEGY':<10} | {'TICKER':<6} | {'ANN RET':<7} | {'SHARPE':<6} | {'MAX DD':<7} | {'NOTES'}")
        print("-" * 75)
        
        strategies = {
            "V1_Base": StrategyV1_Baseline,
        }

        for ticker in self.tickers:
            # Capture Buy & Hold first
            bh = StrategyV1_Baseline(ticker, self.start_date, self.end_date)
            bh.fetch_data()
            bh.data['Signal'] = 1 # Force Buy
            bh.run_backtest()
            self._print_row("Buy&Hold", ticker, bh.metrics)
            
            for name, StratClass in strategies.items():
                try:
                    strat = StratClass(ticker, self.start_date, self.end_date)
                    strat.fetch_data(warmup_years=2)
                    strat.generate_signals()
                    strat.run_backtest()
                    
                    self._print_row(name, ticker, strat.metrics)
                    
                    # Store for portfolio level (optional)
                    self.results.append({
                        'Ticker': ticker,
                        'Strategy': name,
                        'Returns': strat.results['Net_Returns']
                    })
                except Exception as e:
                    print(f"Failed {name} {ticker}: {e}")
            print("-" * 75)

    def _print_row(self, name, ticker, metrics):
        if not metrics: return
        ret = metrics['Total Return']
        # Annualize return approx
        ann_ret = (1 + ret) ** (252 / len(metrics.get('Returns', [1]*252))) - 1 if 'Returns' in metrics else ret
        print(f"{name:<10} | {ticker:<6} | {ret:.1%}   | {metrics['Sharpe Ratio']:.2f}   | {metrics['Max Drawdown']:.1%}   |")

In [165]:
print(f"{'STRATEGY':<12} | {'TICKER':<6} | {'ANN RET':<7} | {'SHARPE':<6} | {'MAX DD':<7} | {'NOTES'}")
print("-" * 79)

class Strategy_Ensemble_Growth(Strategy_Ensemble):
    def __init__(self, ticker, start, end): super().__init__(ticker, start, end, 0.7, 0.3)

strategies = {
    # "V3_Macro": StrategyV3_Macro,
    "V9_Unshackled": StrategyV9_RegimeUnshackled,
    "V12_Macro": StrategyV12_Macro_Switch,
    # "V21_MacroMom": StrategyV21_MacroMomentum,
    # "Ens_Growth": Strategy_Ensemble_Growth,
    # "V23_Turbo": StrategyV23_TurboTrend,
    # "V25_Prec": StrategyV25_PrecisionTrend,
    "AgentV1": AutoRegimeAgent,
    "AgentV2": AutoRegimeAgent_V2,
    "AgentV3": AutoRegimeAgent_V3,
    "AgentV4": AutoRegimeAgent_V4,
    "AgentV6": AutoRegimeAgent_V6_Darwin,
}

# Same Stress Test Basket
tickers = ["NVDA", "JPM", "TSLA", "BABA"]

bench = RobustBenchmark(
    tickers=tickers, 
    start_date="2022-01-01", 
    end_date="2024-12-30"
)

# Manual run loop to handle the specific classes
for ticker in tickers:
    # Buy & Hold
    bh = StrategyV1_Baseline(ticker, bench.start_date, bench.end_date)
    bh.fetch_data()
    bh.data['Signal'] = 1
    bh.run_backtest()
    bench._print_row("Buy&Hold", ticker, bh.metrics)
    
    for name, StratClass in strategies.items():
        try:
            strat = StratClass(ticker, bench.start_date, bench.end_date)
            strat.fetch_data(warmup_years=2)
            strat.generate_signals()
            strat.run_backtest()
            bench._print_row(name, ticker, strat.metrics)
        except Exception as e:
            print(f"Err {name} {ticker}: {e}")
    print("-" * 79)

STRATEGY     | TICKER | ANN RET | SHARPE | MAX DD  | NOTES
-------------------------------------------------------------------------------
Buy&Hold   | NVDA   | 355.4%   | 1.19   | -62.7%   |
V9_Unshackled | NVDA   | 61.9%   | 0.88   | -16.7%   |
V12_Macro  | NVDA   | 159.6%   | 1.57   | -16.7%   |
AgentV1    | NVDA   | 141.6%   | 1.51   | -17.2%   |
AgentV2    | NVDA   | 319.9%   | 1.67   | -17.3%   |
AgentV3    | NVDA   | 261.7%   | 1.56   | -16.3%   |
AgentV4    | NVDA   | 220.9%   | 1.19   | -29.2%   |
AgentV6    | NVDA   | 114.7%   | 1.18   | -19.6%   |
-------------------------------------------------------------------------------
Buy&Hold   | JPM    | 62.1%   | 0.77   | -37.9%   |
V9_Unshackled | JPM    | 39.2%   | 0.70   | -16.7%   |
V12_Macro  | JPM    | 93.1%   | 1.25   | -13.7%   |
AgentV1    | JPM    | 71.8%   | 1.02   | -14.8%   |
AgentV2    | JPM    | 45.2%   | 0.82   | -14.7%   |
AgentV3    | JPM    | 16.7%   | 0.39   | -16.4%   |
AgentV4    | JPM    | 5.8%   | 0.21   | 

In [166]:
# print(f"{'STRATEGY':<15} | {'DESC':<12} | {'ANN RET':<7} | {'SHARPE':<6} | {'MAX DD':<7}")
# print("-" * 65)

# bench_final = RobustBenchmark(tickers=["SHOWDOWN"], start_date="2022-01-01", end_date="2024-12-30")

# # 1. The Ceiling (Crystal Ball)
# max_pot = Strategy_CrystalBall("CRYSTAL", bench_final.start_date, bench_final.end_date)
# max_pot.fetch_data()
# max_pot.generate_signals()
# max_pot.run_backtest()
# bench_final._print_row("Crystal_Ball", "Max Potential", max_pot.metrics)

# # 2. V21 (Base Trend)
# v21 = Strategy_V21_Sector_Portfolio("BASKET", bench_final.start_date, bench_final.end_date)
# v21.fetch_data()
# v21.generate_signals()
# v21.run_backtest()
# bench_final._print_row("V21_Base", "Trend Only", v21.metrics)